<a href="https://colab.research.google.com/github/dimanyen/card-benefits/blob/main/%E4%BF%A1%E7%94%A8%E5%8D%A1%E5%84%AA%E6%83%A0GAI_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install package

In [ ]:
! pip install langchain
! pip install openai
! pip install langchain-cli
! pip install pypdf
! pip install langchain-community
! pip install langchain-openai
! pip install unstructured
! pip install chromadb
! pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.7/987.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.0/372.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.0/135.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.6/328.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

# 建立向量資料庫

In [2]:
! unzip /content/信用卡優惠訊息-20240719T073550Z-001.zip

Archive:  /content/信用卡優惠訊息-20240719T073550Z-001.zip
  inflating: 信用卡優惠訊息/星展銀行.pdf  
  inflating: 信用卡優惠訊息/信用卡優惠懶人包(保費、燃料費、王品、首刷禮、機場貴賓室).pdf  
  inflating: 信用卡優惠訊息/匯豐銀行.pdf  
  inflating: 信用卡優惠訊息/中國信託.pdf  
  inflating: 信用卡優惠訊息/玉山銀行.pdf  
  inflating: 信用卡優惠訊息/台新銀行.pdf  
  inflating: 信用卡優惠訊息/國泰世華.pdf  


In [4]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from google.colab import drive
from langchain.docstore.document import Document
import chromadb
from chromadb.config import Settings
from google.colab import userdata

api_key = userdata.get('OPENAI_API_KEY')

# 掛載Google Drive
# drive.mount('/content/drive', force_remount=True)

# 設置資料夾路徑
folder_path = "/content/信用卡優惠訊息"

# 初始化PDF文件加載器
pdf_loaders = []

# 遍歷資料夾內所有PDF文件
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(folder_path, filename)
        pdf_loaders.append(PyPDFLoader(file_path))

# 確認有無PDF文件
if not pdf_loaders:
    raise FileNotFoundError(f"No PDF files found in directory: {folder_path}")

# 加載所有PDF文件
documents = []
for loader in pdf_loaders:
    documents.extend(loader.load())

# 確認有無加載文檔
if not documents:
    raise ValueError("No documents were loaded.")

# 分割文本
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=30
)
texts = text_splitter.split_documents(documents)

# 確認有無分割文本
if not texts:
    raise ValueError("Text splitting failed.")

# 初始化OpenAI嵌入
embeddings = OpenAIEmbeddings(api_key=api_key)

# 提取文本內容
document_texts = [doc.page_content for doc in texts]

# 確認有無文本內容
if not document_texts:
    raise ValueError("No text content found in documents.")

# 嵌入文檔
document_embeddings = embeddings.embed_documents(document_texts)

# 確認有無嵌入文檔
if not document_embeddings:
    raise ValueError("Document embedding failed.")

# 初始化Chroma客戶端
chroma_client = chromadb.Client(Settings(persist_directory="/content/drive/MyDrive/vector_store"))

# 創建Chroma集合
collection = chroma_client.create_collection(name="document_collection")

# 添加文本和對應的嵌入到Chroma集合
for i, (embedding, text) in enumerate(zip(document_embeddings, document_texts)):
    collection.add(
        ids=[str(i)],
        embeddings=[embedding],
        metadatas=[{"text": text}]
    )

print("Documents have been successfully added to the Chroma vector store.")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Documents have been successfully added to the Chroma vector store.


In [5]:
query = "我時要去日本玩，有麼推薦"
# 將查詢轉換為嵌入向量
query_embedding = embeddings.embed_query(query)

# 在Chroma中搜索相似文檔
results = collection.query(query_embeddings=[query_embedding], n_results=5)

# 獲取相似文檔的文本
similar_texts = [metadata["text"] for metadata in results["metadatas"][0]]

display(similar_texts)

['4.紅檜俱樂部 +竹林亭單饗餐 (日式套餐，原價 NT$1,080+10%) NT$1,600/ 人。  \n條件：  \n限平日使用，限泡湯  \n--- \n##台新銀行 @GoGo悠遊御璽卡  \n#台新 Point(信用卡 )回饋最高 0.3%，無上限 (2024/11/30 止) \n商家：國內一般消費、海外一般消費  \n詳情：一般消費  0.3%無上限。  \n#餐飲 5折(2024/8/31 止) \n商家： FunNow、日本',
 '指定國內餐飲：麥當 勞 \n生活家居： IKEA宜家家居 \n【慶生月】當月壽星限定 ！ \n• 海外知名樂園 10%：東京迪士尼樂園 │大阪環球影城 │東京哈利波特影城 │吉卜\n力公園│名古屋樂高樂園 │韓國愛寶樂園 │首爾樂天世 界 \n• 聚會餐廳 10%：海底撈火鍋 │老四川巴蜀麻辣燙 │Umai屋馬燒肉 │茶六燒肉堂 │\n絵馬別邸 │築間酸菜魚 │平禄壽司 │京星港式飲茶 │龍都酒樓 │築間麻辣火鍋 │紫木\n槿韓國烤肉 │輕井澤鍋物 │秦味館│小食泰泰式料理 │雅室牛排  仁愛圓環 店',
 '商家：國內一般消費  \n詳情：每筆國內一般消費不限金額可享 1%現金回饋，回饋無上限，直接折抵次\n期帳單消費。  \n#現金回饋最高 3%，無上限 (2024/12/31 止) \n商家：日本  \n詳情：使用玉山 JCB日圓極緻卡於日本地區消費享 1.5%現金回饋，同享免收\n1.5%國外交易服務費，合計共 3%回饋。  \n#指定房型 43折(2024/12/31 止、限平日 ) \n商家：台中商旅  \n詳情：入住菁英套房、景觀套房依定價享平日 4.9折、假日 5.4折優惠，行政套',
 '詳情：國內一般消費享 1%現金回饋無上限。  \n#餐飲 5折(2024/8/31 止) \n商家： FunNow、日本  \n詳情：台灣發行之 Visa無限  /商務無限 / 御璽 /商務御璽卡持卡人，預訂東京、\n大阪、沖繩等日本地區指定精選餐廳，透過 FunNow訂位，訂位人數選擇  2人\n2份套餐，於付款頁面輸入優惠碼「 Visa50off 」後，可立即享有線上預訂金額\n50％ 折扣，每月限量 100名。  \n※優惠實際適用餐廳請依 FunNow活動網頁為準。  \n條件：限餐飲',
 '商家： 

In [6]:
from openai import OpenAI
from IPython.display import Markdown, display

client = OpenAI(api_key=api_key)
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "根據我輸入的問題與內容生成結果"},
    {"role": "user", "content": f'問題:{query}\n內容:{similar_texts}'}
  ]
)


display(Markdown(completion.choices[0].message.content))

我推薦你一些日本的熱門景點和活動，讓你可以充分享受你的旅行：

### 1. 東京迪士尼樂園
- **詳細**: 這是一個適合全家人遊玩的樂園。可以體驗到多樣的主題區和景點。
- **優惠**: 使用特定信用卡（如台新GoGo悠遊御璽卡）可享海外樂園10%的優惠。

### 2. 大阪環球影城
- **詳細**: 擁有大量電影主題的景點，非常適合影迷和家庭遊客。
- **優惠**: 使用台新GoGo悠遊御璽卡等特定信用卡享10%的折扣。

### 3. 京都的竹林
- **詳細**: 一片如詩如畫的竹林，適合拍照和散步，感受日本的傳統美。
- **推薦景點**: 竹林亭單饗餐，享受日式套餐。

### 4. 名古屋樂高樂園
- **詳細**: 這裡不僅有適合小朋友的玩樂設施，還有讓成年人感到驚喜的樂高世界。
- **優惠**: 利用台新銀行指定信用卡可享樂園10%折扣。

### 5. 東京的吉卜力公園
- **詳細**: 愛好宮崎駿動畫的必訪之地，能夠重溫那些經典的動畫場景。
- **優惠**: 同樣可以利用台新等信用卡打折。

### 6. 東京的哈利波特影城
- **詳細**: 讓你瞬間穿越到哈利波特的魔法世界，適合拍照和探索。
- **優惠**: 使用指定銀行信用卡可享10%折扣。

### 餐飲推薦
- **FunNow餐飲優惠**: 使用台灣Visa無限、商務無限、御璽等卡預訂日本地區指定精選餐廳，透過FunNow訂位時可享50%折扣。

### 注意事項
- **提前預訂**: 許多景點和餐廳需要提前預訂，尤其是熱門地點。
- **信用卡優惠**: 記得利用不同信用卡提供的優惠，最大限度地減少消費。

希望這些建議對你的日本旅行有所幫助！享受你的旅程吧！